Borrowed some encoder-decoder of dual attention rnn from https://chandlerzuo.github.io/blog/2017/11/darnn

These are seq2seq model which predicts final status sequentially.  
This is not good because we get errors from each prediction step.  

Another thing to note is  
In case prediction_period is larger than 1,
we do not need to infer all information.  
We have information before predicting dates.  
So y market data before predicting date can be used in inference.  

Results are bad for distant prediction.

## Load data

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

In [4]:
import os, itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import sys
try:
    lib_s = '/home/adminuser/public/libs/pytorch_examples/basic_networks'
    sys.path.append(lib_s)
    import generate_model as nets
except:
    lib_s = '/home/bwlee/work/codes/examples/basic_networks'
    sys.path.append(lib_s)
    import generate_model as nets

## Make input data
interest.csv has only closed price.   
On the other hand, zipline needs OLHCV(?) format for each index  
make dummy columns and put 'close' value to them  
make csv file for each index  

In [5]:
f_s0 = 'interest.csv'
df0 = pd.read_csv(f_s0, index_col='date')
df0 = df0.iloc[::-1] # 과거에서 현재로 정렬순서 변경

## prepare data without time confusion
Seperate two actions

In [317]:
# normalized input, norm val, tgt_date, tgt_val, tgt_diff_ratio, 
class Environ():
    def __init__(self):
        self.indices = ['trea3', 'trea5', 'trea10']
        #self.tgt_index = 'trea10'
        self.tgt_index = 'trea3'
        # input window size
        self.input_size = 30
        # prediction is made after algo.pred_period from the present
        self.pred_period = 2

env = Environ() # setting values
mkt_data = {} # object to store data for learning
    
def push_data(env, data, mkt_data):
    """
    read past data and push them to mkt_data
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        if ii_days+1 < env.input_size:
            continue
        elif ii_days+1 == env.input_size:
            env.start_date = data.iloc[ii_days].name
        
        predicting_date = data.iloc[ii_days].name
        # historical data from today to env.input_size behind days
        history = data.iloc[ii_days+1-env.input_size:ii_days+1]
        mean_val = history.mean() 
        max_val = history.max()
        min_val = history.min()
        
        range_val = max_val - min_val
        norm_history = (history-mean_val)/range_val
        mkt_data[predicting_date] = [norm_history, mean_val, range_val, 
                                     None, None, None, None, None]
        
            
def push_tgt(env, data, mkt_data):
    """
    read future data and push them to mkt_data
    future_prediction_date is a future date when model want to predict
    past_predicting_date is a past date when model made a prediction with given information
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        ii_predicting = ii_days - env.pred_period
        predicting_date = data.iloc[ii_predicting].name
        if ii_predicting < 0 or predicting_date < env.start_date:
            continue
        
        past_history = mkt_data[predicting_date][0]
        mean_val = mkt_data[predicting_date][1]
        range_val = mkt_data[predicting_date][2]
        
        past_price = (past_history.iloc[-1]*range_val+mean_val)[env.tgt_index] # 49.6
        # wrong
        #past_priceb = (past_history.iloc[-env.pred_period:]*range_val+mean_val)[env.tgt_index] 
        #print('past_priceb', past_priceb)
        #past_price = (past_history.iloc[-5]*range_val+mean_val)[env.tgt_index] # 56.3
        
        #mkt_data[predicting_date][0] = past_history[:-4]
        
        #past_price = (past_history.iloc[-3]*range_val+mean_val)[env.tgt_index] # 53.0, # 53.0
        #past_price = (past_history.iloc[-13]*range_val+mean_val)[env.tgt_index] # 67.4
        #past_price = (past_history*range_val+mean_val).mean()[env.tgt_index] # 58.4
        #past_price = (past_history*range_val+mean_val)[:-3].mean()[env.tgt_index] # 63.2
        #past_price = (past_history*range_val+mean_val)[:-5].mean()[env.tgt_index] # 63.7
        
        #past_price = (past_history.iloc[-10]*range_val+mean_val)[env.tgt_index] # 66.4

        #past_price = (past_history.iloc[-5:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        #past_price = (past_history.iloc[-3:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        
        #past_price = (past_history.iloc[-10:].mean()*range_val+mean_val)[env.tgt_index] # 55.9
        #past_price = (past_history.iloc[-10:-5].mean()*range_val+mean_val)[env.tgt_index] # 60.4
        
        prediction_date = data.iloc[ii_days].name
        #tgt_price = data.iloc[ii_days][env.tgt_index]
        # tgt should have one more time step than input, because is has initial value
        tgt_price = data.iloc[ii_days-env.input_size:ii_days+1][env.tgt_index]
        tgt_price = np.single(tgt_price)
        
        #tgt_price = (data.iloc[ii_days-1:ii_days+2].mean())[env.tgt_index]
        
        # ignore dates and subtract to get target history
        #tgt_ratio = (tgt_price - np.single(past_price))/np.single(past_price)*10
        tgt_ratio = tgt_price/np.single(past_price)
        #print('diff', tgt_price - np.single(past_priceb))
        #print('ratio', tgt_ratio)
        mkt_data[predicting_date][3] = prediction_date
        mkt_data[predicting_date][4] = tgt_price
        mkt_data[predicting_date][5] = tgt_ratio
        
        mkt_data[predicting_date][6] = np.single(past_price)
        mkt_data[predicting_date][7] = predicting_date

### Push past data

In [318]:
push_data(env, df0, mkt_data)

### Check data are pushed correctly
* no dates should be ahead of key date
* prediction info are not pushed yet

In [30]:
print(list(mkt_data.keys())[:3])

['2000-11-11', '2000-11-13', '2000-11-14']


In [69]:
mkt_data['2000-11-11']

[tensor([[ 0.3000,  0.2721,  0.2940],
         [ 0.1060,  0.1314,  0.1522],
         [ 0.1060,  0.1314,  0.1522],
         [ 0.0910,  0.1240,  0.1522],
         [ 0.1507,  0.1758,  0.1837],
         [ 0.1060,  0.1240,  0.1365],
         [ 0.1284,  0.1240,  0.1050],
         [ 0.1582,  0.1610,  0.1365],
         [ 0.1582,  0.1610,  0.1365],
         [ 0.1881,  0.1980,  0.1759],
         [ 0.1881,  0.2054,  0.1759],
         [ 0.0761,  0.0721,  0.0499],
         [-0.3567, -0.4242, -0.4383],
         [-0.7000, -0.7279, -0.7060],
         [-0.7000, -0.7279, -0.7060]]),
 tensor([7.6640, 7.8663, 8.1433]),
 tensor([0.6700, 0.6750, 0.6350]),
 '2000-11-29',
 tensor([7.1350, 7.1550, 7.2450, 7.2900, 7.2750, 7.2750, 7.2950, 7.4200, 7.3550,
         7.3850, 7.3500, 7.3500, 7.2750, 7.2350, 7.2400]),
 tensor([-0.0834, -0.0556,  0.0695,  0.1320,  0.1112,  0.1112,  0.1390,  0.3127,
          0.2224,  0.2641,  0.2154,  0.2154,  0.1112,  0.0556,  0.0625]),
 7.195,
 '2000-11-11',
 tensor([-0.8881, -0.9333

In [209]:
env.start_date

'2000-11-06'

### Push future data

In [319]:
push_tgt(env, df0, mkt_data)
mkt_data = {key: val for key, val in mkt_data.items() if (val[-1] is not None)}

### CHECK tgt push is correctly made
end value of '2000-11-23' * norm == prediction value 

In [226]:
print(mkt_data['2000-11-11'])
print(mkt_data['2000-11-23'])

[               trea3     trea5    trea10
date                                    
2000-10-26  0.300000  0.272099  0.293963
2000-10-27  0.105970  0.131358  0.152231
2000-10-28  0.105970  0.131358  0.152231
2000-10-30  0.091045  0.123951  0.152231
2000-10-31  0.150746  0.175802  0.183727
2000-11-01  0.105970  0.123951  0.136483
2000-11-02  0.128358  0.123951  0.104987
2000-11-03  0.158209  0.160988  0.136483
2000-11-04  0.158209  0.160988  0.136483
2000-11-06  0.188060  0.198025  0.175853
2000-11-07  0.188060  0.205432  0.175853
2000-11-08  0.076119  0.072099  0.049869
2000-11-09 -0.356716 -0.424198 -0.438320
2000-11-10 -0.700000 -0.727901 -0.706037
2000-11-11 -0.700000 -0.727901 -0.706037, trea3     7.664000
trea5     7.866333
trea10    8.143333
dtype: float64, trea3     0.670
trea5     0.675
trea10    0.635
dtype: float64, '2000-12-05', 7.07, -1.0108074]
[               trea3     trea5    trea10
date                                    
2000-11-07  0.681934  0.687143  0.711795
2000-11-

In [1]:
## Training
### Data preparation

In [320]:
class Index_data(Dataset):
    def __init__(self, in_data, dates):
        """
        :param in_data: dict of (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        """
        # change dataframe to float numpy array
        for key, val in in_data.items():
            if not 'Tensor' in str(type(in_data[key][0])) :
                in_data[key][0] = torch.tensor(in_data[key][0].to_numpy(np.single))
                in_data[key][1] = torch.tensor(in_data[key][1].to_numpy(np.single))
                in_data[key][2] = torch.tensor(in_data[key][2].to_numpy(np.single))
                in_data[key][4] = torch.tensor(in_data[key][4])
                in_data[key][5] = torch.tensor(in_data[key][5])
                
                ## add difference
                diff = in_data[key][0][-1] - in_data[key][0][-5]
                diff2 = in_data[key][0][-1] - in_data[key][0][-7]
                diff3 = in_data[key][0][-1] - in_data[key][0][-10]
                diff4 = in_data[key][0][-1] - in_data[key][0][-3]
                diff5 = in_data[key][0][-3] - in_data[key][0][-7]
                diff6 = in_data[key][0][-3] - in_data[key][0][-10]
                diff7 = in_data[key][0][-5] - in_data[key][0][-7]
                diff8 = in_data[key][0][-5] - in_data[key][0][-10]
                #diff = torch.cat((diff, diff2, diff3), dim=-1)
                diff = torch.cat((diff, diff2, diff3, diff4, diff5, diff6, diff7, diff8), dim=-1)
                in_data[key].append(diff)
        self.in_data = in_data
        self.i2dates = {}
        for ii, date1 in enumerate(dates):
             self.i2dates[ii] = date1
        
    def __getitem__(self, ii_date):
        """
        :return: (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        """
        predicting_date = self.i2dates[ii_date]
        return self.in_data[predicting_date]
    
    def __len__(self):
        return len(self.in_data)

In [321]:
#erase data that do not have prediction target
traintest_dates = sorted(list(mkt_data.keys()))
n_train = len(traintest_dates)*8//10
n_test = len(traintest_dates) - n_train
train_dates = traintest_dates[:n_train]
test_dates = traintest_dates[n_train:]

train_data = {}
test_data = {}
for key, val in mkt_data.items():
    if key in train_dates:
        train_data[key] = val
    elif key in test_dates:
        test_data[key] = val

In [322]:
batch_size = 64
train_dset = Index_data(train_data, train_dates)
test_dset = Index_data(test_data, test_dates)
train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=False)

In [14]:
print(len(test_dset))

1070


In [88]:
test_dset[0]

[tensor([[-0.3948, -0.4402, -0.4924],
         [-0.4127, -0.4342, -0.4758],
         [-0.4307, -0.4704, -0.5015],
         [-0.5139, -0.5590, -0.6024],
         [-0.3228, -0.2934, -0.2630],
         [-0.2105, -0.1324, -0.0520],
         [ 0.1041,  0.1996,  0.2508],
         [ 0.0592,  0.1091,  0.1407],
         [ 0.2951,  0.3022,  0.3058],
         [ 0.1491,  0.1392,  0.1315],
         [ 0.2884,  0.2720,  0.2893],
         [ 0.2727,  0.2700,  0.2783],
         [ 0.3064,  0.3062,  0.3150],
         [ 0.3243,  0.2901,  0.2783],
         [ 0.4861,  0.4410,  0.3976]]),
 tensor([1.6137, 1.7458, 1.9783]),
 tensor([0.4450, 0.4970, 0.5450]),
 '2016-12-15',
 tensor([1.8300, 1.8230, 1.7480, 1.7100, 1.7220, 1.7250, 1.7600, 1.7200, 1.7370,
         1.7310, 1.7090, 1.7350, 1.7160, 1.6620, 1.6450, 1.6950]),
 tensor([ 0.0000, -0.0383, -0.4481, -0.6557, -0.5902, -0.5738, -0.3825, -0.6011,
         -0.5082, -0.5410, -0.6612, -0.5191, -0.6230, -0.9180, -1.0109, -0.7377]),
 1.83,
 '2016-11-24',
 tensor([

In [323]:
from torch.autograd import Variable

class encoder(nn.Module):
    def __init__(self, input_size, hidden_size, T):
        # input size: number of underlying factors (81)
        # T: number of time steps (10)
        # hidden_size: dimension of the hidden state
        super(encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.T = T

        self.lstm_layer = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = 1)
        self.attn_linear = nn.Linear(in_features = 2 * hidden_size + T - 1, out_features = 1)

    def forward(self, input_data):
        # input_data: batch_size * T - 1 * input_size        
        input_weighted = Variable(input_data.data.new(input_data.size(0), self.T - 1, self.input_size).zero_())
        input_encoded = Variable(input_data.data.new(input_data.size(0), self.T - 1, self.hidden_size).zero_())
        # hidden, cell: initial states with dimention hidden_size
        hidden = self.init_hidden(input_data) # 1 * batch_size * hidden_size
        cell = self.init_hidden(input_data)
        # hidden.requires_grad = False
        # cell.requires_grad = False
        for t in range(self.T - 1):
            # Eqn. 8: concatenate the hidden states with each predictor
            x = torch.cat((hidden.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           cell.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           input_data.permute(0, 2, 1)), dim = 2) # batch_size * input_size * (2*hidden_size + T - 1)
            # Eqn. 9: Get attention weights
            #print('size', self.hidden_size, self.T, input_data.size())
            x = self.attn_linear(x.view(-1, self.hidden_size * 2 + self.T - 1)) # (batch_size * input_size) * 1
            attn_weights = F.softmax(x.view(-1, self.input_size)) # batch_size * input_size, attn weights with values sum up to 1.
            # Eqn. 10: LSTM
            weighted_input = torch.mul(attn_weights, input_data[:, t, :]) # batch_size * input_size
            # Fix the warning about non-contiguous memory
            # see https://discuss.pytorch.org/t/dataparallel-issue-with-flatten-parameter/8282
            self.lstm_layer.flatten_parameters()
            _, lstm_states = self.lstm_layer(weighted_input.unsqueeze(0), (hidden, cell))
            hidden = lstm_states[0]
            cell = lstm_states[1]
            # Save output
            input_weighted[:, t, :] = weighted_input
            input_encoded[:, t, :] = hidden
        return input_weighted, input_encoded

    def init_hidden(self, x):
        # No matter whether CUDA is used, the returned variable will have the same type as x.
        return Variable(x.data.new(1, x.size(0), self.hidden_size).zero_()) # dimension 0 is the batch dimension

class decoder(nn.Module):
    def __init__(self, encoder_hidden_size, decoder_hidden_size, T):
        super(decoder, self).__init__()

        self.T = T
        self.encoder_hidden_size = encoder_hidden_size
        self.decoder_hidden_size = decoder_hidden_size

        self.attn_layer = nn.Sequential(nn.Linear(2 * decoder_hidden_size + encoder_hidden_size, encoder_hidden_size),
                                         nn.Tanh(), nn.Linear(encoder_hidden_size, 1))
        self.lstm_layer = nn.LSTM(input_size = 1, hidden_size = decoder_hidden_size)
        self.fc = nn.Linear(encoder_hidden_size + 1, 1)
        self.fc_final = nn.Linear(decoder_hidden_size + encoder_hidden_size, 1)

        self.fc.weight.data.normal_()

    def forward(self, input_encoded, y_history):
        # input_encoded: batch_size * T - 1 * encoder_hidden_size
        # y_history: batch_size * (T-1)
        # Initialize hidden and cell, 1 * batch_size * decoder_hidden_size
        hidden = self.init_hidden(input_encoded)
        cell = self.init_hidden(input_encoded)
        # hidden.requires_grad = False
        # cell.requires_grad = False
        for t in range(self.T - 1):
            # Eqn. 12-13: compute attention weights
            ## batch_size * T * (2*decoder_hidden_size + encoder_hidden_size)
            x = torch.cat((hidden.repeat(self.T - 1, 1, 1).permute(1, 0, 2),
                           cell.repeat(self.T - 1, 1, 1).permute(1, 0, 2), input_encoded), dim = 2)
            x = F.softmax(self.attn_layer(x.view(-1, 2 * self.decoder_hidden_size + self.encoder_hidden_size
                                                )).view(-1, self.T - 1)) # batch_size * T - 1, row sum up to 1
            # Eqn. 14: compute context vector
            context = torch.bmm(x.unsqueeze(1), input_encoded)[:, 0, :] # batch_size * encoder_hidden_size
            if t < self.T - 1:
                # Eqn. 15
                y_tilde = self.fc(torch.cat((context, y_history[:, t].unsqueeze(1)), dim = 1)) # batch_size * 1
                # Eqn. 16: LSTM
                self.lstm_layer.flatten_parameters()
                _, lstm_output = self.lstm_layer(y_tilde.unsqueeze(0), (hidden, cell))
                hidden = lstm_output[0] # 1 * batch_size * decoder_hidden_size
                cell = lstm_output[1] # 1 * batch_size * decoder_hidden_size
        # Eqn. 22: final output
        y_pred = self.fc_final(torch.cat((hidden[0], context), dim = 1))
        return y_pred
        
    def predict(self, input_encoded, y_history):
        # input_encoded: batch_size * T - 1 * encoder_hidden_size
        # y_history: batch_size * (T-1)
        # this is used in cases i-(input-1)+1 <= i-env.period
        # Initialize hidden and cell, 1 * batch_size * decoder_hidden_size
        with torch.no_grad():
            hidden = self.init_hidden(input_encoded)
            cell = self.init_hidden(input_encoded)
            # y_online : replace history, we do not past values when online predicting
            # : batch_size * (T-1)
            y_online = torch.zeros(hidden.size()[1], self.T-1).to(device)
            #print('y_online', y_online.size())
            for t in range(self.T - 1):
                # Eqn. 12-13: compute attention weights
                ## batch_size * T * (2*decoder_hidden_size + encoder_hidden_size)
                x = torch.cat((hidden.repeat(self.T - 1, 1, 1).permute(1, 0, 2),
                               cell.repeat(self.T - 1, 1, 1).permute(1, 0, 2), input_encoded), dim = 2)
                x = F.softmax(self.attn_layer(x.view(-1, 2 * self.decoder_hidden_size + self.encoder_hidden_size
                                                    )).view(-1, self.T - 1)) # batch_size * T - 1, row sum up to 1
                # Eqn. 14: compute context vector
                context = torch.bmm(x.unsqueeze(1), input_encoded)[:, 0, :] # batch_size * encoder_hidden_size                
                if t < self.T - 1:
                    # Eqn. 15
                    #print('size', 'context', context.size(), 'online', 
                    #      y_online[:, t].unsqueeze(1).size())
                    y_tilde = self.fc(torch.cat((context, y_online[:, t].unsqueeze(1)), dim = 1)) # batch_size * 1
                    #print('y_tilde', y_tilde.size())
                    # Eqn. 16: LSTM
                    self.lstm_layer.flatten_parameters()
                    y_output, lstm_output = self.lstm_layer(y_tilde.unsqueeze(0), (hidden, cell))
                    hidden = lstm_output[0] # 1 * batch_size * decoder_hidden_size
                    cell = lstm_output[1] # 1 * batch_size * decoder_hidden_size

                    if t+1 >= self.T-1:
                        break
                        
                    y_pred = self.fc_final(torch.cat((hidden[0], context), dim = 1))
                    #print('y_pred', y_pred.size(), 'y_tiled', y_tilde.size())
                    # in case, seq2seq out is already known, we can use them
                    if env.input_size>= env.pred_period+2:
                        if t+1<env.input_size-env.pred_period-1: 
                            y_online[:, t+1] = y_history[:, t+1].squeeze()
                        else:
                            y_pred = self.fc_final(torch.cat((hidden[0], context), dim = 1))
                            y_online[:, t+1] = y_pred.squeeze()
            # Eqn. 22: final output
            y_pred = self.fc_final(torch.cat((hidden[0], context), dim = 1))
            return y_pred

    def init_hidden(self, x):
        return Variable(x.data.new(1, x.size(0), self.decoder_hidden_size).zero_())


In [324]:
class Predictor_rnn(nets.Net):
    def __init__(self, downnet=None, dim_hiddens=[200, 200], 
                 loss=None, optimizer=None, device='cuda:2',
                 dim_input=100, bidirectional=True, time_step=15, diff=None):
        """
        net is consists of [embed, rnn, downnet]
        :param downnet: define downstream job
        """
        super(Predictor_rnn, self).__init__(loss=loss, device=device)
        
        encoder_dim_hidden, decoder_dim_hidden = dim_hiddens
        self.encoder = encoder(dim_input, encoder_dim_hidden, time_step)
        self.decoder = decoder(encoder_dim_hidden, decoder_dim_hidden, time_step)
            
        if downnet is None:
            self.downnet = nets.get_MLP([decoder_dim_hidden, decoder_dim_hidden*2, 1], 
                                        dropout=0.35, end=True)
        else:
            self.downnet = downnet
        
        self.init_weights()
        
        self.encoder = self.encoder.to(device)
        self.decoder = self.decoder.to(device)
        self.downnet = self.downnet.to(device)
        
        parms = list(self.encoder.parameters())
        parms += list(self.decoder.parameters())
        parms += list(self.downnet.parameters())
        self.optimizer = optimizer(parms)
        
        self.diff = diff
            
        
    def set_train(self):
        self.encoder.train()
        self.decoder.train()
        self.downnet.train()
        
    def set_eval(self):
        self.decoder.eval()
        self.decoder.eval()
        self.downnet.eval()
    
    def init_weights(self):
        # check is there a way to initialize lstm parameters?
        for ii, layer in enumerate(self.downnet):
            if 'Linear' in str(layer):
                #torch.nn.init.xavier_uniform_(self.downnet[ii].weight)
                torch.nn.init.xavier_normal_(self.downnet[ii].weight)

    def forward(self, x, diff=None, tgt_history=None):
        input_weighted, input_encoded = self.encoder(x)
        out = self.decoder(input_encoded, tgt_history) 
        return out.view(-1)
    
    def predict(self, x, diff=None, tgt_history=None):
        input_weighted, input_encoded = self.encoder(x)
        out = self.decoder.predict(input_encoded, tgt_history) 
        return out.view(-1)

    def run_eval(self, data):
        self.set_eval()
        loss = 0
        outs = None
        tgts = None
        with torch.no_grad():
            for i_batch, data_batch in enumerate(data):
                data_in = data_batch[0][:,1:]
                mean_val = data_batch[1]
                range_val = data_batch[2]
                pred_date = data_batch[3]
                tgts_in = data_batch[5]
                tgt_history = tgts_in[:,:-1]
                tgt = tgts_in[:,-1]
                if self.diff is not None:
                    diff = data_batch[-1]
                    diff = diff.to(self.device)
                
                data_in = data_in.to(self.device)
                tgt = tgt.to(self.device)
                tgt_history = tgt_history.to(self.device)
                
                if self.diff is not None:
                    #out = self.forward(data_in, diff, tgt_history)
                    out = self.predict(data_in, diff, tgt_history)
                else:
                    #out = self.forward(data_in, None, tgt_history)
                    out = self.predict(data_in, None, tgt_history)
                    
                #print('loss', loss, self.loss)
                loss += self.loss(out, tgt).cpu().numpy()
                out = out.cpu().numpy()
                #print('loss===', loss)
                tgt = tgt.cpu().numpy()
                #print('tgt', tgt.shape)
                if outs is None:
                    outs = out
                    tgts = tgt
                else:
                    outs = np.concatenate((outs, out), axis=0)
                    tgts = np.concatenate((tgts, tgt), axis=0)
        loss /= 1.0*(i_batch+1)
        print('evaluate', 'loss', loss, 'accuracy : define function')
        return outs, tgts, loss
    
    def run_batch(self, i_batch, data_batch):
        self.optimizer.zero_grad()
        data_in = data_batch[0][:,1:]
        mean_val = data_batch[1]
        range_val = data_batch[2]
        pred_date = data_batch[3]
        tgts_in = data_batch[5]
        tgt_history = tgts_in[:,:-1]
        tgt = tgts_in[:,-1]
        if self.diff is not None:
            diff = data_batch[-1]
            diff = diff.to(self.device)
            
        data_in = data_in.to(self.device)
        tgt = tgt.to(self.device)
        tgt_history = tgt_history.to(self.device)
        
        if self.diff is not None:
            out = self.forward(data_in, diff, tgt_history)
        else:
            out = self.forward(data_in, None, tgt_history)
        
        loss = self.loss(out, tgt)
        loss.backward()
        self.optimizer.step()
        return loss.detach().cpu().item()

In [333]:
dim_input = train_dset[0][0].shape[-1]
time_step = train_dset[0][0].shape[0]
dim_hidden = 100
#downnet = nets.get_MLP([dim_hidden+3*3, dim_hidden, 1], 
# Add difference variables
downnet = nets.get_MLP([dim_hidden, dim_hidden, 1], 
#downnet = nets.get_MLP([dim_hidden+3*8, dim_hidden, 1], 
                        dropout=0.35, end=True)
#bidownnet = nets.get_MLP([dim_hidden*2+3*8, dim_hidden*2, 1], 
#                        dropout=0.0, end=True)
loss = nn.MSELoss() # which combines logsoftmax and nll loss
optimizer = optim.Adam
device = torch.device("cuda:2")

In [334]:
predictor = Predictor_rnn(loss=loss, optimizer=optimizer, 
                          device=device, dim_input=dim_input, 
                          dim_hiddens=[dim_hidden, dim_hidden], 
                          downnet=None, bidirectional=False, diff=None, 
                         time_step=env.input_size)
                          #dim_hidden=dim_hidden, downnet=bidownnet, bidirectional=True)

In [335]:
predictor.run_train(n_epoch=30, data=train_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 loss 0.12860228629685494
eval_train 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


evaluate loss 0.0005477339006724086 accuracy : define function
epoch 1 loss 0.00038118369628869884
eval_train evaluate loss 0.00035803300830702397 accuracy : define function
epoch 2 loss 0.00036544392103461356
eval_train evaluate loss 0.00035814328206904624 accuracy : define function
epoch 3 loss 0.00036232815508091293
eval_train evaluate loss 0.000354326714773718 accuracy : define function
epoch 4 loss 0.0003606828447403525
eval_train evaluate loss 0.00036836467872707366 accuracy : define function
epoch 5 loss 0.00036032572032452853
eval_train evaluate loss 0.0003542932983637273 accuracy : define function
epoch 6 loss 0.000360294001493186
eval_train evaluate loss 0.00035258263081082826 accuracy : define function
epoch 7 loss 0.0003569716222344467
eval_train evaluate loss 0.00037151206686133657 accuracy : define function
epoch 8 loss 0.00037142807624733715
eval_train evaluate loss 0.00036581343738362193 accuracy : define function
epoch 9 loss 0.000362729712140941
eval_train evaluate lo

In [340]:
rets = {}
tgts = None
preds = None
with torch.no_grad():
    for data_batch in test_loader:
        data_in = data_batch[0][:,1:]
        tgts_in = data_batch[5]
        tgt_history = tgts_in[:,:-1]
        tgt = tgts_in[:,-1]
        #diff = data_batch[-1]
        #diff = diff.to(device)
        
        data_in = data_in.to(device)
        tgt_history = tgt_history.to(device)
        
        tgt = tgt.numpy()
        #out = predictor.forward(data_in).cpu().numpy()
        out = predictor.predict(data_in, None, tgt_history).cpu().numpy()
        if preds is None:
            preds = out
            tgts = tgt
        else:
            preds = np.concatenate((preds, out), axis=0)
            tgts = np.concatenate((tgts, tgt), axis=0)
            
        prediction_dates = data_batch[3]
        past_prices = data_batch[6]
        predicting_dates = data_batch[7]
        for ii, date1 in enumerate(prediction_dates):
            out1 = out[ii]
            #out_orig = out1/10*past_prices[ii] + past_prices[ii]
            out_orig = out1*past_prices[ii]
            rets[date1] = {'predicting_date': predicting_dates[ii],
                          'pred': out_orig,
                           'past': past_prices[ii],
                           'tgt': data_batch[4][ii][-1]}
            
preds = preds.reshape(-1)
tgts = tgts.reshape(-1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [341]:
list(rets.keys()).index('2021-01-04')

996

In [342]:
limit = 999
sum1 = np.sum((preds[limit:] > 1) & (tgts[limit:] > 1))
sum2 = np.sum((preds[limit:] < 1) & (tgts[limit:] < 1))
print(sum1, np.sum(tgts[limit:]>1))
print(sum2, np.sum(tgts[limit:]<1))
print(len(preds[limit:]), sum1+sum2, 1.0*(sum1+sum2)/len(preds[limit:]))

0 37
29 29
68 29 0.4264705882352941


In [343]:
sum1 = np.sum((preds > 1) & (tgts > 1))
sum2 = np.sum((preds < 1) & (tgts < 1))
print(sum1, np.sum(tgts>1))
print(sum2, np.sum(tgts<1))
print(len(preds), sum1+sum2, 1.0*(sum1+sum2)/len(preds))

0 492
545 545
1067 545 0.5107778819119025


In [22]:
rets_2021 = {key: val for key, val in rets.items() if key>'2021-01-01'}
rows = list(rets_2021['2021-01-04'].keys())
cols = list(rets_2021.keys())

In [27]:
rets_2021b = {}
for col in cols:
    for row in rows:
        rets_2021b.setdefault(row, {})
        val = rets_2021[col][row]
        if type(val) is not str:
            rets_2021b[row][col] = rets_2021[col][row].item()        
        else:
            rets_2021b[row][col] = rets_2021[col][row]